In [9]:
import os
import sys
root_directory = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(root_directory,"src"))
DATA_PATH = os.getcwd()+"/../../data/"

import boto3
from botocore.exceptions import ClientError
s3 = boto3.resource('s3')
BUCKET_NAME = 'idol-timetable'

from backend_functions import s3access

In [16]:
my_bucket = s3.Bucket(BUCKET_NAME)
print(my_bucket.name)
my_bucket.upload_file(
    Filename='../../data_tmp/timetable_sample/通常フェス/20240827_エンドレスサマー_LIVE.png',
    Key='test/testfile.png'
)


idol-timetable


In [14]:
my_bucket.download_file(
    Key='test/testfile.png',
    Filename='data/testfile.png'
)

In [7]:
#権限のないS3へのアクセスは失敗する
my_bucket = s3.Bucket('idol-fes-imetable')
print(my_bucket.name)
my_bucket.upload_file(
    Filename='../../data_tmp/timetable_sample/通常フェス/20240827_エンドレスサマー_LIVE.png',
    Key='test/testfile.png'
)

idol-fes-imetable


S3UploadFailedError: Failed to upload ../../data_tmp/timetable_sample/通常フェス/20240827_エンドレスサマー_LIVE.png to idol-fes-imetable/test/testfile.png: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist

In [ ]:
def upload_directory_to_s3(local_directory, s3_prefix=""):
    """
    local_directory配下のファイルを再帰的に列挙し、
    s3_prefixを先頭としたKey名でS3へアップロードする。
    """
    for root, dirs, files in os.walk(local_directory):
        for file_name in files:
            # ローカルファイルの絶対パス
            local_path = os.path.join(root, file_name)

            # S3上でのファイルパスを生成
            # 例: root= /path/to/local_directory/subdir, file_name= file.txt
            #     relative_path = subdir/file.txt
            #     s3_key = my-prefix/subdir/file.txt
            relative_path = os.path.relpath(local_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/")

            print(f"Uploading {local_path} to s3://{BUCKET_NAME}/{s3_key}")
            my_bucket.upload_file(local_path, s3_key)

In [34]:
local_dir = "../../data/master"
prefix = "master"
s3access.upload_directory_to_s3(local_dir, prefix)

Uploading ../../data/master\idolname_embedding_data.csv to s3://idol-timetable/master/idolname_embedding_data.csv
Uploading ../../data/master\idolname_latest.csv to s3://idol-timetable/master/idolname_latest.csv
Uploading ../../data/master\idolname_tobeupdated.csv to s3://idol-timetable/master/idolname_tobeupdated.csv
Uploading ../../data/master\master_version.json to s3://idol-timetable/master/master_version.json
Uploading ../../data/master\projects_master.csv to s3://idol-timetable/master/projects_master.csv


In [20]:
local_dir = "../../data/projects"
prefix = "projects"
s3access.upload_directory_to_s3(local_dir, prefix)

Uploading ../../data/projects\atjam2024\project_info.json to s3://idol-timetable/projects/atjam2024/project_info.json
Uploading ../../data/projects\atjam2024\event_1\ライブ\raw.png to s3://idol-timetable/projects/atjam2024/event_1/ライブ/raw.png
Uploading ../../data/projects\atjam2024\event_1\ライブ\raw_cropped.png to s3://idol-timetable/projects/atjam2024/event_1/ライブ/raw_cropped.png
Uploading ../../data/projects\atjam2024\event_1\ライブ\stage_0.json to s3://idol-timetable/projects/atjam2024/event_1/ライブ/stage_0.json
Uploading ../../data/projects\atjam2024\event_1\ライブ\stage_0.png to s3://idol-timetable/projects/atjam2024/event_1/ライブ/stage_0.png
Uploading ../../data/projects\atjam2024\event_1\ライブ\stage_1.json to s3://idol-timetable/projects/atjam2024/event_1/ライブ/stage_1.json
Uploading ../../data/projects\atjam2024\event_1\ライブ\stage_1.png to s3://idol-timetable/projects/atjam2024/event_1/ライブ/stage_1.png
Uploading ../../data/projects\atjam2024\event_1\ライブ\stage_2.json to s3://idol-timetable/projects/a

In [30]:
def download_prefix_from_s3(s3_prefix, local_directory, bucket_name=BUCKET_NAME):
    """
    指定したS3バケットとプレフィックスにあるオブジェクトを、
    ローカルフォルダに一括ダウンロードする。
    """
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')

    for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        # オブジェクトが存在しない場合、Contentsキーが無いことがある
        if 'Contents' not in page:
            continue

        for obj in page['Contents']:
            key = obj['Key']
            # S3上でディレクトリプレースホルダの場合、キーが`xxx/`で終わることがあるのでスキップ
            if key.endswith('/'):
                continue
            filename = key.split("/")[-1]

            # ローカルに保存するためのパスを生成
            # プレフィックス部分を削除し、残りをファイルパスにする #最低限ファイル名は保持する
            relative_path = key[len(s3_prefix):].lstrip('/')
            if len(relative_path)<len(filename):
                relative_path = filename
            local_path = os.path.join(local_directory, relative_path)

            # ダウンロード先のディレクトリがなければ作成
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            print(f"Downloading s3://{BUCKET_NAME}/{key} to {local_path}")
            my_bucket.download_file(key, local_path)

In [8]:
prefix = "projects/大青春祭/p"
local_download_dir = "data/projetcts/大青春祭/"

s3access.download_prefix_from_s3(prefix, local_download_dir)

In [10]:
s3access.download_s3_object("master/master_version.json", os.path.join(DATA_PATH, "master"), "master_version_s3.json")


[Info] Found existing file. Renaming: c:\Users\kkoaz\Documents\projects\product\timetable-detect\timetable-detect\test\notebook/../../data/master\master_version_s3.json -> c:\Users\kkoaz\Documents\projects\product\timetable-detect\timetable-detect\test\notebook/../../data/master\master_version_s3.json.old_e50f6b0e-1b34-4aeb-975d-9e5069b67734
Downloaded s3://idol-timetable/master/master_version.json to c:\Users\kkoaz\Documents\projects\product\timetable-detect\timetable-detect\test\notebook/../../data/master\master_version_s3.json
[Info] Download succeeded. Removing backup file: c:\Users\kkoaz\Documents\projects\product\timetable-detect\timetable-detect\test\notebook/../../data/master\master_version_s3.json.old_e50f6b0e-1b34-4aeb-975d-9e5069b67734


In [24]:
prefix = "projects/大青春祭/"
local_download_dir = "data/projetcts/大青春祭/"

s3access.download_prefix_from_s3(prefix, local_download_dir)


In [11]:
pj_name = "大青春祭"
s3access.get_project_data(pj_name)